In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load the dataset from CSV
def load_dataset(csv_file):
    df = pd.read_csv(csv_file)
    return df['input_text'].tolist(), df['target_text'].tolist()

In [ ]:
# Data preprocessing
def preprocess_data(input_texts, target_texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(input_texts + target_texts)
    
    input_sequences = tokenizer.texts_to_sequences(input_texts)
    target_sequences = tokenizer.texts_to_sequences(target_texts)
    
    max_sequence_length = max([len(seq) for seq in input_sequences])
    
    input_sequences_padded = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
    target_sequences_padded = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')
    
    vocab_size = len(tokenizer.word_index) + 1
    
    return input_sequences_padded, target_sequences_padded, vocab_size, max_sequence_length, tokenizer

In [ ]:
# Define the LSTM model
def create_model(vocab_size, embedding_dim, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
        LSTM(units=64, return_sequences=True),
        LSTM(units=64),
        Dense(vocab_size, activation='softmax')
    ])
    return model

In [ ]:
# Train the model
def train_model(model, X_train, y_train, X_val, y_val, num_epochs, batch_size):
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val))

In [ ]:
# Evaluate the model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Test Loss:', loss)
    print('Test Accuracy:', accuracy)


In [ ]:
# Perform inference
def predict_next_word(model, input_text, tokenizer):
    input_seq = tokenizer.texts_to_sequences([input_text])[0]
    input_seq = pad_sequences([input_seq], maxlen=len(input_seq), padding='post')
    predicted_probs = model.predict(input_seq)[0]
    predicted_index = np.argmax(predicted_probs)
    predicted_word = tokenizer.index_word[predicted_index]
    return predicted_word

In [ ]:
# Load the dataset
input_texts, target_texts = load_dataset('output.csv')

In [ ]:
# Preprocess the data
X, y, vocab_size, max_sequence_length, tokenizer = preprocess_data(input_texts, target_texts)

In [ ]:
# Split the dataset into training, validation, and testing sets
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Define hyperparameters
embedding_dim = 100
num_epochs = 10
batch_size = 32

# Create and train the model
model = create_model(vocab_size, embedding_dim, max_sequence_length)
train_model(model, X_train, y_train, X_val, y_val, num_epochs, batch_size)

In [ ]:
# Evaluate the model
evaluate_model(model, X_test, y_test)

In [ ]:
input_text = "Aaj mausam"
predicted_word = predict_next_word(model, input_text, tokenizer)
print("Predicted next word:", predicted_word)
